In [1]:
import os
import librosa
import numpy as np
import pandas as pd
import tensorflow_hub as hub
from pycaret.classification import *
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, mean_squared_error, r2_score)
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict

In [2]:
# Caminhos para as pastas de áudio
pasta_ruido = r'../audios/audios_treinamento/classe_ruido'
pasta_barco = r'../audios/audios_treinamento/classe_barco'

# Função para extrair features dos áudios
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    features = {
        'chroma_stft': librosa.feature.chroma_stft(y=y, sr=sr).mean(),
        'chroma_cqt': librosa.feature.chroma_cqt(y=y, sr=sr).mean(),
        'chroma_cens': librosa.feature.chroma_cens(y=y, sr=sr).mean(),
        'rmse': librosa.feature.rms(y=y).mean(),
        'spectral_centroid': librosa.feature.spectral_centroid(y=y, sr=sr).mean(),
        'spectral_bandwidth': librosa.feature.spectral_bandwidth(y=y, sr=sr).mean(),
        'spectral_contrast': librosa.feature.spectral_contrast(y=y, sr=sr).mean(),
        'spectral_flatness': librosa.feature.spectral_flatness(y=y).mean(),
        'spectral_rolloff': librosa.feature.spectral_rolloff(y=y, sr=sr).mean(),
        'rolloff': librosa.feature.spectral_rolloff(y=y, sr=sr).mean(),
        'zero_crossing_rate': librosa.feature.zero_crossing_rate(y).mean(),
        'mfcc1': librosa.feature.mfcc(y=y, sr=sr)[0].mean(),
        'mfcc2': librosa.feature.mfcc(y=y, sr=sr)[1].mean(),
        'mfcc3': librosa.feature.mfcc(y=y, sr=sr)[2].mean(),
        # 'mfcc4': librosa.feature.mfcc(y=y, sr=sr)[3].mean(),
        # 'mfcc5': librosa.feature.mfcc(y=y, sr=sr)[4].mean(),
        # 'mfcc6': librosa.feature.mfcc(y=y, sr=sr)[5].mean(),
        # 'mfcc7': librosa.feature.mfcc(y=y, sr=sr)[6].mean(),
        # 'mfcc8': librosa.feature.mfcc(y=y, sr=sr)[7].mean(),
        # 'mfcc9': librosa.feature.mfcc(y=y, sr=sr)[8].mean(),
        # 'mfcc10': librosa.feature.mfcc(y=y, sr=sr)[9].mean(),
        # 'mfcc11': librosa.feature.mfcc(y=y, sr=sr)[10].mean(),
        # 'mfcc12': librosa.feature.mfcc(y=y, sr=sr)[11].mean(),
        # 'mfcc13': librosa.feature.mfcc(y=y, sr=sr)[12].mean(),
        #'tempo': librosa.beat.tempo(y=y, sr=sr)[0]
    }
    return features

# Função para processar as pastas e adicionar os dados ao DataFrame
def verifica_pastas(folder_path, label):
    data = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_path.endswith('.wav'):
            features = extract_features(file_path)
            features['label'] = label
            data.append(features)
    return data

# Processar as duas pastas sendo cada uma uma classe
dados_ruido = verifica_pastas(pasta_ruido, 1)
dados_barco = verifica_pastas(pasta_barco, 2)

# Combinar os dados e criar um DataFrame
data = dados_ruido + dados_barco
df = pd.DataFrame(data)

# Salvar em um arquivo CSV
df.to_csv("CSV\\features_audios_classificados.csv", index=False)

In [5]:


dados = pd.read_csv("CSV\\features_audios_classificados.csv")
X = dados.drop(columns=['label'])
y = dados['label']

rf_model = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                                  criterion='gini', max_depth=None, max_features='sqrt',
                                  max_leaf_nodes=None, max_samples=None,
                                  min_impurity_decrease=0.0, min_samples_leaf=1,
                                  min_samples_split=2, min_weight_fraction_leaf=0.0,
                                  n_estimators=100, n_jobs=-1, oob_score=False, 
                                  random_state=123, verbose=0, warm_start=False)

# Avaliar o modelo usando validação cruzada
cv_scores = cross_val_score(rf_model, X, y, cv=5, scoring='accuracy')
print(f'Cross-validation scores: {cv_scores}')
print(f'Mean cross-validation score: {np.mean(cv_scores)}')

# Obter previsões de validação cruzada
y_pred_cv = cross_val_predict(rf_model, X, y, cv=5)

# Gerar o relatório de classificação para a validação cruzada
print("Relatório de Classificação para Random Forest Classifier (Validação Cruzada):")
print(classification_report(y, y_pred_cv))

Cross-validation scores: [0.85714286 1.         1.         0.92307692 0.84615385]
Mean cross-validation score: 0.9252747252747252
Relatório de Classificação para Random Forest Classifier (Validação Cruzada):
              precision    recall  f1-score   support

           1       0.92      0.89      0.91        27
           2       0.93      0.95      0.94        39

    accuracy                           0.92        66
   macro avg       0.92      0.92      0.92        66
weighted avg       0.92      0.92      0.92        66



In [6]:
# Dividir o dataset em treino e teste para uma avaliação adicional
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=123)

rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
print("Relatório de Classificação para Random Forest Classifier (Conjunto de Teste):")
print(classification_report(y_test, y_pred))

Relatório de Classificação para Random Forest Classifier (Conjunto de Teste):
              precision    recall  f1-score   support

           1       0.93      0.93      0.93        14
           2       0.95      0.95      0.95        19

    accuracy                           0.94        33
   macro avg       0.94      0.94      0.94        33
weighted avg       0.94      0.94      0.94        33



In [7]:
clf1 = setup(data=dados, target='label', session_id=123)
save_model(rf_model, 'Modelo_RFC')

,Description,Value
0,Session id,123
1,Target,label
2,Target type,Binary
3,Target mapping,"1: 0, 2: 1"
4,Original data shape,"(66, 15)"
5,Transformed data shape,"(66, 15)"
6,Transformed train set shape,"(46, 15)"
7,Transformed test set shape,"(20, 15)"
8,Numeric features,14
9,Preprocess,True


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['chroma_stft', 'chroma_cqt',
                                              'chroma_cens', 'rmse',
                                              'spectral_centroid',
                                              'spectral_bandwidth',
                                              'spectral_contrast',
                                              'spectral_flatness',
                                              'spectral_r...
                  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                         class_weight=None, criterion='gini',
                                         max_depth=None,